In [1]:
import requests
import time
import re
import pandas as pd
import numpy as np
import random
from bs4 import BeautifulSoup

In [2]:
projects=pd.DataFrame(columns=['kickstarter_page','funding', 'campaign_dates', 'backers', 'category'])

In [3]:
#1-8000 are available
page_numbers = np.random.randint(1,high=8000,size=5)
for page_num in page_numbers:
    url = 'https://www.kicktraq.com/archive/?page='+str(page_num)
    print('url is ' + url)
    kickstarter_page = []
    funding = []
    campaign_dates = []
    backers = []
    categories = []
    
    response = requests.get(url)
    
    while True:
        if response.status_code == 200:
                break
        else:
            response = requests.get(url)
    
    page = response.text
    soup = BeautifulSoup(page,'lxml')
    
    
    all_projects_on_page = soup.find_all(class_="project first odd closed")
    all_projects_on_page = all_projects_on_page + soup.find_all(class_="project odd closed")
    all_projects_on_page = all_projects_on_page + soup.find_all(class_="project closed")

    for project in all_projects_on_page: 
        #if statement tests to see if funding info is available. If it is, we include the project. If not, we don't!
        funding_info = project.find(class_='project-details').find(text=re.compile('Funding: .\d*.\d* of .\d*.\d*'))
        if funding_info != None:
            funding.append(str(funding_info).replace('$',''))

            kickstarter_page.append(str(project.find('h2').find('a')['href']))
            campaign_dates.append(str(project.find(class_='project-details').find(text=re.compile('Dates: \w* \d*\w* \-\> \w* \d*\w* \(\d*\)'))))
            backers.append(str(project.find(class_='project-details').find(text=re.compile('Backers: \d*'))))
            categories.append(str(project.find(class_='project-cat').find('a').text))
            d = {'kickstarter_page': kickstarter_page,'funding': funding, 'campaign_dates': campaign_dates, 'backers': backers,'category': categories}
            page_df = pd.DataFrame(d)

    projects = pd.concat([projects, page_df])


    time.sleep(10+random.uniform(0,2))


url is https://www.kicktraq.com/archive/?page=234
url is https://www.kicktraq.com/archive/?page=2636
url is https://www.kicktraq.com/archive/?page=7178
url is https://www.kicktraq.com/archive/?page=7664
url is https://www.kicktraq.com/archive/?page=1958


In [8]:
projects.kickstarter_page.unique().shape

(60,)

In [ ]:
projects = projects.drop_duplicates

In [ ]:
projects['description_length'] = 0
projects['n_pledges'] = 0
projects['pledge_level_min'] = 0
projects['pledge_level_max'] = 0
projects['pledge_level_stddev'] = 0
projects['currency'] = '$'

In [ ]:
def get_kickstarter_data(proj_series):
    url = 'https://www.kickstarter.com/' + proj_series.kickstarter_page
    print(url)    
    response = requests.get(url)
    while True:
        if response.status_code == 200:
                break
        else:
            response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,'lxml')

    pledges = soup.find_all(class_='pledge__info')
    
    n_pledges = len(pledges)
    
    pledge_amounts=[]
    for pledge in pledges:
        pledge_amount = str(pledge.find(class_='money').text.replace('$',''))
        currency = ''
        #while the first character in pledge_amount is not an integer
        while not (re.match('\d',pledge_amount[0])):
            currency = currency + (pledge_amount[0])
            pledge_amount = pledge_amount[1:]
        if currency == '':
            currency = '$'
        proj_series['currency'] = currency
        pledge_amounts.append(int(pledge_amount.replace(',','')))

    descript_paragraphs = soup.find(class_="full-description js-full-description responsive-media formatted-lists").find_all('p')
    description_words = 0
    for par in descript_paragraphs:
        description_words += len(str(par.text).split())
        
    proj_series.description_length = description_words
    proj_series.n_pledges = n_pledges
    proj_series.pledge_level_min = min(pledge_amounts)
    proj_series.pledge_level_max = max(pledge_amounts)
    proj_series.pledge_level_stddev = np.std(pledge_amounts)
    
    return proj_series
        
    time.sleep(2+random.uniform(0,2))

In [ ]:
projects = projects.apply(get_kickstarter_data, axis=1)

In [ ]:
projects.head()

In [ ]:
import pickle
with open('kickstarter_data_raw.pickle', 'wb') as f:
    pickle.dump(projects,f)

In [ ]:
#selenium

In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import os

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.binary_location = '/usr/bin/google-chrome'

chromedriver = "/bin/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver


driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")


In [10]:
for page_num in page_numbers:
    url = 'https://www.kicktraq.com/archive/?page='+str(page_num)
    driver.get(url)
    time.sleep(10)